1. Создаем таблицы по условию

create table public.customers(
	customer_ID integer primary key
	,first_name text
	,last_name text
	,gender text
	,DOB date 
	,job_title text
	,job_industry_category text
	,wealth_segment text not null
	,deceased_indicator text not null
	,owns_car text not null
	,address  varchar not null
	,postcode integer not null
	,state text not null
	,country text not null
	,property_valuation integer not null
);

create table public.products(
    id serial primary key
	,product_ID integer
	,brand text
	,product_line text
	,product_class text
	,product_size text
	,list_price varchar
	,standard_cost varchar
);

create table public.orders(
	order_ID integer primary key
	,customer_ID integer
	,order_date date
	,online_order varchar
	,order_status varchar
);

create table public.order_items(
	order_item_id integer primary key
	,order_ID integer
	,product_ID integer
	,quantity integer
	,item_list_price_at_sale integer
	,item_standard_cost_at_sale integer
);

2. Загружаем данные в таблицу customers

<img src="cust.png" width="600">

3. Загружаем данные в таблицу orders

<img src="order.png" width="600">

4. Загружаем данные в таблицу products

<img src="prod.png" width="600">

5. Загружаем данные в таблицу order_items

<img src="ord_it.png" width="600">

Далее выполняем запросы. 

-- 1 пункт
select distinct p.brand
from public.products p
where nullif(p.standard_cost, '')::numeric > 1500 and 1000<= (
	select sum(oi.quantity)
	from public.order_items oi
	where oi.product_ID = p.product_ID
);

<img src="1_.png" width="600">

-- 2 пункт
select 
	ord.order_date,
	count(*) as podt_onl_ord,
	count(distinct ord.customer_ID) as uniq_cust
from public.orders ord
where 
	ord.order_date between '2017-04-01' and '2017-04-09'
	and ord.order_status = 'Approved' 
	and ord.online_order = 'True'
group by ord.order_date;

<img src="2_.png" width="600">

-- 3 пункт
select cs.job_title
from public.customers cs
where cs.job_industry_category = 'IT' 
and cs.job_title like 'Senior%'
and age(cs.dob) > interval '35 years'
union all
select cs.job_title
from public.customers cs
where cs.job_industry_category = 'Financial Services' 
and cs.job_title like 'Lead%'
and age(cs.dob) > interval '35 years';

<img src="3_.png" width="600">

-- 4 пункт
--Бренды, купленные Financial Services
select distinct p.brand
from public.customers c
join public.orders o on c.customer_ID = o.customer_ID
join public.order_items oi on o.order_ID = oi.order_ID
join public.products p on oi.product_ID = p.product_ID
where c.job_industry_category = 'Financial Services'
except
--Бренды, купленные IT
select distinct p.brand
from public.customers c
join public.orders o on c.customer_ID = o.customer_ID
join public.order_items oi on o.order_ID = oi.order_ID
join public.products p on oi.product_ID = p.product_ID
where c.job_industry_category = 'IT';

<img src="4_.png" width="600">

-- 5 пункт
select 
    c.customer_id, 
    c.first_name, 
    c.last_name
from public.customers c
join public.orders o on c.customer_id = o.customer_id
join public.order_items oi on o.order_id = oi.order_id
join public.products p on oi.product_id = p.product_id
where 
    o.online_order = 'True'
    and o.order_status = 'Approved'
    and c.deceased_indicator = 'N'
    and p.brand in ('Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles')
    and c.property_valuation > (
        select avg(c2.property_valuation)
        from public.customers c2
        where c2.state = c.state
    )
group by c.customer_id, c.first_name, c.last_name
order by sum(oi.quantity) desc
limit 10;

<img src="5_.png" width="600">

-- 6 пункт
select c.customer_id, c.first_name, c.last_name
from public.customers c
where 
    c.owns_car = 'Yes'
    and c.wealth_segment <> 'Mass Customer'
    and c.customer_id not in (
        select o.customer_id
        from public.orders o
        where 
            o.online_order = 'True'
            and o.order_status = 'Approved'
            and o.order_date >= CURRENT_DATE - INTERVAL '1 year'
    );

<img src="6_.png" width="600">

-- 7 пункт
with top5_road as (
    select product_id
    from public.products
    where product_line = 'Road'
    order by NULLIF(list_price, '')::numeric desc
    limit 5
)
select c.customer_id, c.first_name, c.last_name
from public.customers c
join public.orders o on c.customer_id = o.customer_id
join public.order_items oi on o.order_id = oi.order_id
join top5_road t on oi.product_id = t.product_id
where c.job_industry_category = 'IT'
group by c.customer_id, c.first_name, c.last_name
having count(distinct oi.product_id) >= 2;

<img src="7_.png" width="600">

-- 8 пункт
-- IT
select 
    c.customer_id,
    c.first_name,
    c.last_name,
    c.job_industry_category
from public.customers c
join public.orders o on c.customer_id = o.customer_id
join public.order_items oi on o.order_id = oi.order_id
where 
    c.job_industry_category = 'IT'
    and o.order_status = 'Approved'
    and o.order_date between '2017-01-01' and '2017-03-01'
group by c.customer_id, c.first_name, c.last_name, c.job_industry_category
having  
    count(o.order_id) >= 3
    and sum(oi.quantity * oi.item_list_price_at_sale) > 10000

union

-- Health
select 
    c.customer_id,
    c.first_name,
    c.last_name,
    c.job_industry_category
from public.customers c
join public.orders o on c.customer_id = o.customer_id
join public.order_items oi on o.order_id = oi.order_id
where 
    c.job_industry_category = 'Health'
    and o.order_status = 'Approved'
    and o.order_date between '2017-01-01' and '2017-03-01'
group by c.customer_id, c.first_name, c.last_name, c.job_industry_category
having 
    count(o.order_id) >= 3
    and sum(oi.quantity * oi.item_list_price_at_sale) > 10000;

<img src="8_.png" width="600">